In [1]:
import torch

print(torch.__version__)
print(*torch.__config__.show().split("\n"), sep="\n")

1.5.0
PyTorch built with:
  - C++ Version: 199711
  - MSVC 191627039
  - Intel(R) Math Kernel Library Version 2020.0.0 Product Build 20191125 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v0.21.1 (Git Hash 7d2fd500bc78936d1d648ca713b901012f470dbc)
  - OpenMP 200203
  - CPU capability usage: AVX2
  - Build settings: BLAS=MKL, BUILD_TYPE=Release, CXX_FLAGS=/DWIN32 /D_WINDOWS  /GR  /w /EHa /bigobj -openmp -DNDEBUG -DUSE_FBGEMM, PERF_WITH_AVX=1, PERF_WITH_AVX2=1, PERF_WITH_AVX512=1, USE_CUDA=0, USE_EXCEPTION_PTR=1, USE_GFLAGS=OFF, USE_GLOG=OFF, USE_MKL=ON, USE_MKLDNN=ON, USE_MPI=OFF, USE_NCCL=OFF, USE_NNPACK=OFF, USE_OPENMP=ON, USE_STATIC_DISPATCH=OFF, 



In [2]:
from testnet import Config, Backbone
with torch.no_grad():
    conf = Config()
    model = Backbone(conf.net_depth, conf.drop_ratio, conf.net_mode, conf.net_output)
    model.load_state_dict(torch.load('model.pth', map_location='cpu'))
    model.eval()
    sample = torch.rand(1,3,112,112)
    traced_script_module = torch.jit.trace(model, sample)
    traced_script_module.save("trace_model.pt")
    print(traced_script_module)

Backbone(
  original_name=Backbone
  (input_layer): Sequential(
    original_name=Sequential
    (0): Conv2d(original_name=Conv2d)
    (1): BatchNorm2d(original_name=BatchNorm2d)
    (2): PReLU(original_name=PReLU)
  )
  (output_layer): Sequential(
    original_name=Sequential
    (0): BatchNorm2d(original_name=BatchNorm2d)
    (1): Dropout(original_name=Dropout)
    (2): Flatten(original_name=Flatten)
    (3): Linear(original_name=Linear)
    (4): BatchNorm1d(original_name=BatchNorm1d)
  )
  (body): ModuleList(
    original_name=ModuleList
    (0): bottleneck_IR_SE(
      original_name=bottleneck_IR_SE
      (shortcut_layer): MaxPool2d(original_name=MaxPool2d)
      (res_layer): Sequential(
        original_name=Sequential
        (0): BatchNorm2d(original_name=BatchNorm2d)
        (1): Conv2d(original_name=Conv2d)
        (2): PReLU(original_name=PReLU)
        (3): Conv2d(original_name=Conv2d)
        (4): BatchNorm2d(original_name=BatchNorm2d)
        (5): SEModule(
          origi

In [3]:
torch.manual_seed(0)
sample = torch.rand(1,3,112,112)

In [4]:
with torch.no_grad():
    conf = Config()
    model_test = Backbone(conf.net_depth, conf.drop_ratio, conf.net_mode, conf.net_output)
    model_test.load_state_dict(torch.load('model.pth', map_location='cpu'))
    model_test.eval()
    test_result = model_test(sample)
    print (test_result)

tensor([[-3.1600e-02,  4.2761e-02,  2.5042e-02,  6.5828e-02,  7.0371e-02,
          5.0582e-02, -1.1572e-02, -3.2238e-02,  2.6101e-02, -1.4062e-02,
         -4.5667e-02,  6.4934e-02,  9.5757e-02, -2.0643e-02, -2.9308e-02,
         -1.3256e-02, -4.7344e-02,  4.3644e-02,  6.9615e-03,  1.8289e-02,
         -2.7115e-03,  1.7935e-02,  6.5289e-02, -1.3841e-02, -5.8035e-02,
         -3.1505e-02,  3.1199e-03,  1.3652e-02, -5.2991e-02, -1.0606e-01,
         -4.9601e-03, -1.3562e-01,  5.3160e-02, -6.4687e-02, -3.0776e-02,
          3.3088e-02, -1.3708e-02, -2.2573e-02, -1.2025e-02,  2.0663e-03,
         -4.4643e-02, -4.0790e-02, -2.6682e-02, -4.1074e-02, -2.2767e-02,
          3.3183e-02,  1.8416e-02, -5.4665e-02,  1.4356e-03,  1.8318e-02,
         -2.5179e-02, -1.1563e-01,  3.8396e-02, -6.7564e-02,  3.8611e-02,
          4.7622e-02,  2.0180e-02, -5.7676e-02,  5.1862e-03, -8.1917e-03,
         -5.8405e-02,  2.7923e-02,  6.9302e-02,  2.6093e-02, -4.1418e-02,
         -5.3674e-02, -3.7643e-02, -7.

In [5]:
with torch.no_grad():
    model_trace_test = torch.jit.load("trace_model.pt")
    model_trace_test.eval()
    test_trace_result = model_trace_test(sample)
    print (test_trace_result)


tensor([[-3.1600e-02,  4.2761e-02,  2.5042e-02,  6.5828e-02,  7.0371e-02,
          5.0582e-02, -1.1572e-02, -3.2238e-02,  2.6101e-02, -1.4062e-02,
         -4.5667e-02,  6.4934e-02,  9.5757e-02, -2.0643e-02, -2.9308e-02,
         -1.3256e-02, -4.7344e-02,  4.3644e-02,  6.9615e-03,  1.8289e-02,
         -2.7115e-03,  1.7935e-02,  6.5289e-02, -1.3841e-02, -5.8035e-02,
         -3.1505e-02,  3.1199e-03,  1.3652e-02, -5.2991e-02, -1.0606e-01,
         -4.9601e-03, -1.3562e-01,  5.3160e-02, -6.4687e-02, -3.0776e-02,
          3.3088e-02, -1.3708e-02, -2.2573e-02, -1.2025e-02,  2.0663e-03,
         -4.4643e-02, -4.0790e-02, -2.6682e-02, -4.1074e-02, -2.2767e-02,
          3.3183e-02,  1.8416e-02, -5.4665e-02,  1.4356e-03,  1.8318e-02,
         -2.5179e-02, -1.1563e-01,  3.8396e-02, -6.7564e-02,  3.8611e-02,
          4.7622e-02,  2.0180e-02, -5.7676e-02,  5.1862e-03, -8.1917e-03,
         -5.8405e-02,  2.7923e-02,  6.9302e-02,  2.6093e-02, -4.1418e-02,
         -5.3674e-02, -3.7643e-02, -7.